In [1]:
import numpy as np

#initialize functions and load previous solutions
#1D analytical solutions for PB equation in spherical coordinates
#output potential and potential gradient
def an_sol(x,a,rel_perm,tK,cinf,zval,zeta,gradflag=0):
    echarge=1.602e-19 #elementary charge [C]
    perm0=8.85e-12 #vacuum permittivity [F/m]
    kB=1.381e-23 #Boltzmann's constant [J/K]
    kA=6.022e23 #Avogadro constant [1/mol]

    debye_len=np.sqrt(perm0*rel_perm*kB*tK/2/(zval*echarge)**2/kA/cinf)
    print('DEBYE LENGTH IS:',debye_len*1e9,'nm')
    
    if gradflag:
        zeta=zeta/(1/debye_len+1/a)/perm0/rel_perm
    
    u=zeta*a*np.exp((a-x)/debye_len)/x
    du=-u*(1/debye_len+1/x) #du/dr
    return u,du

#compute gradient of scalar field u
#output du is complex if input u is complex
def gradient(xn,yn,zn,u):
    #input u on nodes
    #output gradient in elements
    du=np.zeros(3) #du is constant inside an element
    Je=np.zeros((4,4))
    
    Je[0,:]=1
    Je[1,:]=xn #x-coordinates of four nodes in an element
    Je[2,:]=yn
    Je[3,:]=zn
    invJe=np.linalg.inv(Je)
    
    du[0]=np.sum(u[:]*invJe[:,1])
    du[1]=np.sum(u[:]*invJe[:,2])
    du[2]=np.sum(u[:]*invJe[:,3])
    
    #distance was not scaled in this script
    #du=du*kd #do not forget distance scaling
    return du

#next time save u and du together
u3=np.load('sphere_s41_u3_4.npy')

mesh_prefix='sphere'

#load mesh
print('Reading %s.1.node'%mesh_prefix)
nodes=np.genfromtxt(mesh_prefix+'.1.node',skip_header=1,skip_footer=1,usecols=(1,2,3))
node_flags=np.genfromtxt(mesh_prefix+'.1.node',skip_header=1,skip_footer=1,usecols=5,dtype='int')

print('Reading %s.1.ele'%mesh_prefix)
elements=np.genfromtxt(mesh_prefix+'.1.ele',skip_header=1,usecols=(1,2,3,4),dtype='int')
zones=np.genfromtxt(mesh_prefix+'.1.ele',skip_header=1,usecols=5,dtype='int')

print('Reading %s.1.face'%mesh_prefix)
faces=np.genfromtxt(mesh_prefix+'.1.face',skip_header=1,usecols=(1,2,3),dtype='int')
face_flags=np.genfromtxt(mesh_prefix+'.1.face',skip_header=1,usecols=4,dtype='int')

#adjust indices to start from zero
elements=elements-1
faces=faces-1

# #translate z coordinate
# print('Translating Z coordinate')
# nodes[:,2]=-(np.power(10,-nodes[:,2]/1000*3)-1) #problem dependent
dist1=np.sqrt(nodes[:,0]**2+nodes[:,1]**2+nodes[:,2]**2)
mask=dist1>5.0
dist2=np.zeros_like(dist1)
dist2[mask]=10**((dist1[mask]-5)/50*2)+5-1

print(min(dist1[mask]),max(dist1[mask]))
print(min(dist2[mask]),max(dist2[mask]))
nodes[mask,0]=nodes[mask,0]/dist1[mask]*dist2[mask]
nodes[mask,1]=nodes[mask,1]/dist1[mask]*dist2[mask]
nodes[mask,2]=nodes[mask,2]/dist1[mask]*dist2[mask]

#scale nodes from meter to nano-meter
nodes=nodes*1e-9

nnode=len(nodes)
nelem=len(elements)
nface=len(faces)
print('THE NUMBER OF NODES IS: %d'%nnode)
print('THE NUMBER OF ELEMENTS IS: %d'%nelem)
print('THE NUMBER OF FACES IS: %d'%nface)

#compute middle point of each elements (efficiency to be improved)
nelem=len(elements[:,0])
midpoints=np.zeros((nelem,3))
for i in range(0,nelem):
    for j in range(0,3):
        midpoints[i,j]=.25*sum(nodes[elements[i,:],j])

#compute analytical solution
dist=np.sqrt(midpoints[:,0]**2+midpoints[:,1]**2+midpoints[:,2]**2)
ua,dua_len=an_sol(dist,a=5e-9,rel_perm=78.5,tK=298,cinf=100,zval=1,zeta=0.01,gradflag=1)
print('')

dua=np.zeros((nelem,3))
dua[:,0]=dua_len*midpoints[:,0]/dist
dua[:,1]=dua_len*midpoints[:,1]/dist
dua[:,2]=dua_len*midpoints[:,2]/dist

due3=np.zeros((nelem,3))
for i in range(nelem):
    sctr=elements[i,:4]
    xn=nodes[sctr,0]
    yn=nodes[sctr,1]
    zn=nodes[sctr,2]
    due3[i,:]=gradient(xn,yn,zn,np.real(u3[sctr,2]))
    
print('Done')

Reading sphere.1.node
Reading sphere.1.ele
Reading sphere.1.face
5.000058817654048 87.4685657822283
5.000005417320811 1993.4939888608171
THE NUMBER OF NODES IS: 24157
THE NUMBER OF ELEMENTS IS: 135851
THE NUMBER OF FACES IS: 17578
DEBYE LENGTH IS: 0.9617530004493543 nm

Done


In [3]:
#modified on 6/23/2021
#setup a in main.py with shape changed from (nelem,) to (nnode,)
#setup a in main.py with shape changed from (nelem,) to (nnode,)
#replaced perm with rel_perm
#changed cinf unit from m^-3 to mol*m^-3 by removing kA from its value

#modified on 8/18/2021
#changed g and q from element based to node based

mesh_prefix='sphere'
outfile='sphere_u3.npz'

#set physical constants and properties
echarge=1.602e-19 #elementary charge [C]
perm0=8.85e-12 #vacuum permittivity [F/m]
kB=1.381e-23 #Boltzmann's constant [J/K]
kA=6.022e23 #Avogadro constant [1/mol]

rel_perm=80 #relative permittivity of bulk electrolyte [F/m]
tK=293 #room/ambient temperature [K]
cinf=1 #ion concentration in bulk electrolyte [mol/m^3]
zval=1 #ion valence in bulk electrolyte
# cinf_pos=0.01 #ion concentration of positive charges
# cinf_neg=0.01 #concentration of negative charges
# zval_pos=1 #positive ion valence
# zval_neg=1 #negative ion valence

rel_perm=78.5
tK=298
cinf=100
zval=1

diffusion=2*1e-6**2/1e-3 #Diffusion coefficient of water at 20 deg. C is 2*(1e-6*m)^2/(1e-3*s)
mobility=5e-8 #mobility of sodium cation [m^2/(Vs)]

#load mesh
print('Reading %s.1.node'%mesh_prefix)
nodes=np.genfromtxt(mesh_prefix+'.1.node',skip_header=1,skip_footer=1,usecols=(1,2,3))
node_flags=np.genfromtxt(mesh_prefix+'.1.node',skip_header=1,skip_footer=1,usecols=5,dtype='int')

print('Reading %s.1.ele'%mesh_prefix)
elements=np.genfromtxt(mesh_prefix+'.1.ele',skip_header=1,usecols=(1,2,3,4),dtype='int')
zones=np.genfromtxt(mesh_prefix+'.1.ele',skip_header=1,usecols=5,dtype='int')

print('Reading %s.1.face'%mesh_prefix)
faces=np.genfromtxt(mesh_prefix+'.1.face',skip_header=1,usecols=(1,2,3),dtype='int')
face_flags=np.genfromtxt(mesh_prefix+'.1.face',skip_header=1,usecols=4,dtype='int')

#adjust indices to start from zero
elements=elements-1
faces=faces-1

# #translate z coordinate
# print('Translating Z coordinate')
# nodes[:,2]=-(np.power(10,-nodes[:,2]/1000*3)-1) #problem dependent
dist1=np.sqrt(nodes[:,0]**2+nodes[:,1]**2+nodes[:,2]**2)
mask=dist1>5.0
dist2=np.zeros_like(dist1)
dist2[mask]=10**((dist1[mask]-5)/50*2)+5-1

print(min(dist1[mask]),max(dist1[mask]))
print(min(dist2[mask]),max(dist2[mask]))
nodes[mask,0]=nodes[mask,0]/dist1[mask]*dist2[mask]
nodes[mask,1]=nodes[mask,1]/dist1[mask]*dist2[mask]
nodes[mask,2]=nodes[mask,2]/dist1[mask]*dist2[mask]

#scale nodes from meter to nano-meter
nodes=nodes*1e-9

nnode=len(nodes)
nelem=len(elements)
nface=len(faces)
print('THE NUMBER OF NODES IS: %d'%nnode)
print('THE NUMBER OF ELEMENTS IS: %d'%nelem)
print('THE NUMBER OF FACES IS: %d'%nface)

#set boolean indicies for volume elements, S1 nodes, and S2 elements
mask_e=zones>=3 #elements in zones 3 and 4
# mask_d=node_flags==1 #boolean node indices True for Dirichlet nodes
# mask_s=face_flags==1 #boolean face indices True for third kind surface elements
mask_d=((node_flags==1)|(node_flags==2))|(node_flags==20) #outer boundary may need to be updated
mask_s=np.zeros_like(face_flags,dtype=bool) #empty for this case
# mask_s=face_flags==20 #interface between zone 1 and 2
nind_e=np.unique(elements[mask_e].flatten(order='C'))

#initialize scalar PDE coefficients
print('Setting PDE coefficients')
# cx=perm0*rel_perm*np.ones(nelem)
# cy=perm0*rel_perm*np.ones(nelem)
# cz=perm0*rel_perm*np.ones(nelem)

# cx[(zones==1)|(zones==2)]=perm0*4.5
# cy[(zones==1)|(zones==2)]=perm0*4.5
# cz[(zones==1)|(zones==2)]=perm0*4.5

cx=np.zeros(nelem)
cy=np.zeros(nelem)
cz=np.zeros(nelem)

cx[zones>=3]=diffusion
cy[zones>=3]=diffusion
cz[zones>=3]=diffusion

alphax=np.zeros(nelem)
alphay=np.zeros(nelem)
alphaz=np.zeros(nelem)

alphax[zones>=3]=-mobility*dua[zones>=3,0]
alphay[zones>=3]=-mobility*dua[zones>=3,1]
alphaz[zones>=3]=-mobility*dua[zones>=3,2]

betax=np.zeros(nelem) #unused so far
betay=np.zeros(nelem) #unused so far
betaz=np.zeros(nelem) #unused so far

gammax=np.zeros(nelem,dtype=complex)
gammay=np.zeros(nelem,dtype=complex)
gammaz=np.zeros(nelem,dtype=complex)

gammax[zones>=3]=mobility*cinf*np.exp(echarge*ua[zones>=3]/kB/tK)*due3[zones>=3,0]
gammay[zones>=3]=mobility*cinf*np.exp(echarge*ua[zones>=3]/kB/tK)*due3[zones>=3,1]
gammaz[zones>=3]=mobility*cinf*np.exp(echarge*ua[zones>=3]/kB/tK)*due3[zones>=3,2]

a=np.zeros(nnode,dtype=complex)
f=np.zeros(nnode)
# a='pb' #will be setup in main.py with shape (nnode,)
# f='pb' #will be setup in main.py with shape (nnode,)
a[nind_e]=1j*0.1

g=np.zeros(nnode) #length of ns for essential entries
q=np.zeros(nnode) #length of ns for essential entries
s=np.zeros(nnode) #length of nd for essential entries

#set Dirichlet B.C on solid interface
#for monovalent ions (zval=1), we can use linear PB equation for phi_e<<26mV
#for divalent ions (zval=2), we can use linear PB equation for phi_e<<13 mv
#s[node_flags==10]=0.1 #phi_e=0.1V
s[mask_d]=np.real(u3[mask_d,0])
#nind_s=np.unique(faces[face_flags==20].flatten(order='C'))
#g[nind_s]=0.01 #surface charge density [C/m^2]

#initialize vector PDE coefficients
cx33=np.zeros((nelem,3,3)) #PDE term coefficient
cy33=np.zeros((nelem,3,3)) #PDE term coefficient
cz33=np.zeros((nelem,3,3)) #PDE term coefficient

cx33[zones>=3,0,0]=perm0*rel_perm
cx33[zones>=3,1,1]=perm0*rel_perm
cx33[zones>=3,2,2]=perm0*rel_perm

cx33[zones<=2,0,0]=perm0*4.5
cx33[zones<=2,1,1]=perm0*4.5
cx33[zones<=2,2,2]=perm0*4.5

cy33=np.copy(cx33)
cz33=np.copy(cx33)

alphax33=np.zeros((nelem,3,3)) #PDE term coefficient
alphay33=np.zeros((nelem,3,3)) #PDE term coefficient
alphaz33=np.zeros((nelem,3,3)) #PDE term coefficient

betax33=np.zeros((nelem,3,3)) #PDE term coefficient
betay33=np.zeros((nelem,3,3)) #PDE term coefficient
betaz33=np.zeros((nelem,3,3)) #PDE term coefficient

gammax33=np.zeros((nelem,3,3)) #PDE term coefficient
gammay33=np.zeros((nelem,3,3)) #PDE term coefficient
gammaz33=np.zeros((nelem,3,3)) #PDE term coefficient

a33=np.zeros((nelem,3,3),dtype=complex) #PDE term coefficient
f3=np.zeros((nelem,3),dtype=complex) #source term coefficient
g3=np.zeros((nnode,3)) #Neumann boundary condition coefficient
q33=np.zeros((nnode,3,3)) #Neumann boundary condition coefficient
s3=np.zeros((nnode,3)) #Dirichlet boundary condition coefficient

nind_s=np.unique(faces[face_flags==20].flatten(order='C'))
g3[nind_s,0]=0.01 #surface charge density [C/m^2]
g3[nind_s,1]=0.01 #surface charge density [C/m^2]
g3[nind_s,2]=0.01 #surface charge density [C/m^2]

#save PDE coefficients to a file
print('Saving PDE coefficients to %s'%outfile)
np.savez(outfile,nodes=nodes,elements=elements,faces=faces,
         node_flags=node_flags,zones=zones,face_flags=face_flags,
         mask_e=mask_e,mask_d=mask_d,mask_s=mask_s,
         cx=cx,cy=cy,cz=cz,
         alphax=alphax,alphay=alphay,alphaz=alphaz,
         betax=betax,betay=betay,betaz=betaz,
         gammax=gammax,gammay=gammay,gammaz=gammaz,
         a=a,f=f,g=g,q=q,s=s,
         cx33=cx33,cy33=cy33,cz33=cz33,
         alphax33=alphax33,alphay33=alphay33,alphaz33=alphaz33,
         betax33=betax33,betay33=betay33,betaz33=betaz33,
         gammax33=gammax33,gammay33=gammay33,gammaz33=gammaz33,
         a33=a33,f3=f3,g3=g3,q33=q33,s3=s3,
         rel_perm=rel_perm,tK=tK,cinf=cinf,zval=zval)
print('')


Reading sphere.1.node
Reading sphere.1.ele
Reading sphere.1.face
5.000058817654048 87.4685657822283
5.000005417320811 1993.4939888608171
THE NUMBER OF NODES IS: 24157
THE NUMBER OF ELEMENTS IS: 135851
THE NUMBER OF FACES IS: 17578
Setting PDE coefficients
Saving PDE coefficients to sphere_u3.npz

